<a href="https://colab.research.google.com/github/Gauthampharish/OCR/blob/main/contour_based_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install keras-tuner

In [10]:

!pip install emnist

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import emnist


In [12]:

from emnist import extract_training_samples
train_images, train_labels = extract_training_samples('byclass')

In [13]:

from emnist import extract_test_samples
test_images, test_labels = extract_test_samples('byclass')


In [14]:

train_images = tf.keras.utils.normalize(train_images, axis = 1)
test_images = tf.keras.utils.normalize(test_images, axis = 1)

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

In [15]:

rotation_range_val = 15
width_shift_val = 0.10
height_shift_val = 0.10

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range = rotation_range_val,
                             width_shift_range = width_shift_val,
                             height_shift_range = height_shift_val)

In [17]:
train_datagen.fit(train_images.reshape(train_images.shape[0], 28, 28, 1))

In [18]:
test_datagen = ImageDataGenerator()
test_datagen.fit(test_images.reshape(test_images.shape[0], 28, 28, 1))
test_datagen

In [19]:
# Define the model
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Reshape((-1, 64)))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(62, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [20]:

# Import necessary libraries
from kerastuner.tuners import RandomSearch
import tensorflow as tf

# Set up the tuner with a temporary directory in Colab
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=2,
    executions_per_trial=2,
    directory='/content/drive/MyDrive/colab_temp',  # You can change this directory as needed
    project_name='helloworld'
)


In [21]:
from keras.utils import to_categorical

train_labels_one_hot = to_categorical(train_labels, num_classes=62)
test_labels_one_hot = to_categorical(test_labels, num_classes=62)

In [ ]:
# Search for the best hyperparameters
tuner.search(train_datagen.flow(train_images, train_labels_one_hot, batch_size=1024),
             validation_data=test_datagen.flow(test_images, test_labels_one_hot, batch_size=32),
             epochs=8)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


In [22]:
from tensorflow.keras.models import load_model

# Replace 'saved_model_path' with the path to your saved model directory
saved_model_path = '/content/your_model.h5'

# Load the model
model = load_model(saved_model_path)

In [ ]:
# Build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)

In [ ]:
# Train the model with the training data
model.fit(train_datagen.flow(train_images, train_labels_one_hot, batch_size=1024), epochs=15)


In [ ]:

# Evaluate the model with the testing data
model.evaluate(test_datagen.flow(test_images, test_labels_one_hot, batch_size=32))

In [ ]:
from tensorflow.keras.models import load_model, save_model

# Save the model
model.save("your_model.h5")

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('/content/new.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

characters = []
for contour in contours:
    # Get bounding box for each contour
    x, y, w, h = cv2.boundingRect(contour)

    # Extract the character
    character = gray[y:y+h, x:x+w]

    # Resize to a standard size (e.g., 28x28)
    character = cv2.resize(character, (28, 28))

    characters.append(character)

print(f'Number of characters: {len(characters)}')

# Save the characters as numpy arrays
characters_array = []
for i, char in enumerate(characters):
    # Convert the character to a numpy array
    char_array = np.array(char)

    # Append the character array to the list of characters
    characters_array.append(char_array)

# Convert the list of characters to a numpy array
characters_array = np.array(characters_array)



In [ ]:
def index_to_char(index):
    if index < 10:
        return str(index)
    elif index < 36:
        return chr(index - 10 + ord('A'))
    else:
        return chr(index - 36 + ord('a'))

# Predict the character for each image
for i, char_img in enumerate(characters_array):
    # Preprocess the image
    char_img = char_img.reshape(1, 28, 28, 1)
    char_img = char_img.astype('float32')
    char_img /= 255

    # Predict the character
    prediction = model.predict([char_img])
    predicted_index = np.argmax(prediction)
    predicted_char = index_to_char(predicted_index)

    print(f'Character {i}: {predicted_char}')